In [ ]:
import threading

class Environmental_Patches_Generator(tf.keras.utils.Sequence) :
  
    def __init__(self, obs_ids, labels, batch_size) :z
        self.obs_ids = obs_ids
        self.labels = labels
        self.batch_size = batch_size
        #self.gps = gps
        #self.extractor = extractor
        #print("INIT")
        # to make the generator thread safe 
        self.lock = threading.Lock()

    def __len__(self) :
        return (np.ceil(len(self.obs_ids) / float(self.batch_size))).astype(int)
  
    # returns one batch
    def __getitem__(self, idx) :
        X_batch = list()
        y_batch = list()
        X_env_batch = list()

        #print("ONE BATCH")
        for i in range(idx * self.batch_size, (idx+1) * self.batch_size):
            if i >= len(self.obs_ids): break
            
            rgb, near_ir, landcover, altitude = load_patch(self.obs_ids[i], DATA_PATH, data='all')
            ni = near_ir.reshape(256, 256, 1)
            lc = landcover.reshape(256, 256, 1)
            alt = altitude.reshape(256, 256, 1)

            patch = np.concatenate((rgb, ni, lc, alt), axis=2)

  
            #cs = MinMaxScaler()
            #print("PATCH GENERATOR")
            #print((df_env.loc[self.obs_ids[i]].values).shape)
            #print(cs.fit_transform(df_env.loc[self.obs_ids[i]].values).shape)
            #k = input()
            X_env_batch.append(df_env.loc[self.obs_ids[i]].values)
            #X_env_batch.append(df_env[self.obs_ids[i], :])
            #X_env_batch.append(cs.fit_transform(df_env.loc[self.obs_ids[i]].values.reshape(-1,1)))
            X_batch.append(patch)
            y_batch.append(self.labels[i])

            



        with self.lock:
        
            
            #return {'input_1': np.asarray(X_batch), 'input_2': np.asarray(X_env_batch)}, np.asarray(np.array(y_batch))
            #return np.asarray(X_batch), np.array(y_batch)
            return (np.asarray(X_batch), np.asarray(X_env_batch)), np.array(y_batch)

In [ ]:
import threading

class Test_Patches_Generator(tf.keras.utils.Sequence) :
  
    def __init__(self, obs_ids, batch_size) :
        self.obs_ids = obs_ids
        self.batch_size = batch_size
        
        # to make the generator thread safe 
        self.lock = threading.Lock()

    def __len__(self) :
        return (np.ceil(len(self.obs_ids) / float(self.batch_size))).astype(int)
  
    # returns one batch
    def __getitem__(self, idx) :
        X_batch = list()
        y_batch = list()

        for i in range(idx * self.batch_size, (idx+1) * self.batch_size):
            if i >= len(self.obs_ids): break
            
            patch = load_patch(self.obs_ids[i], DATA_PATH, data='rgb')
            X_batch.append(patch[0])

        with self.lock:
            return np.asarray(X_batch)

#### Pretrained Patches

In [ ]:
import threading

class Pretrained_Patches_Generator(tf.keras.utils.Sequence) :
  
    def __init__(self, obs_ids, labels, batch_size) :
        self.obs_ids = obs_ids
        self.labels = labels
        self.batch_size = batch_size
        #self.gps = gps
        #self.extractor = extractor
        #print("INIT")
        # to make the generator thread safe 
        self.lock = threading.Lock()

    def __len__(self) :
        return (np.ceil(len(self.obs_ids) / float(self.batch_size))).astype(int)
  
    # returns one batch
    def __getitem__(self, idx) :
        X_batch_rgb = list()
        X_batch_images = list()
        y_batch = list()
        X_env_batch = list()

        #print("ONE BATCH")
        for i in range(idx * self.batch_size, (idx+1) * self.batch_size):
            if i >= len(self.obs_ids): break
            
            rgb, near_ir, landcover, altitude = load_patch(self.obs_ids[i], DATA_PATH, data='all')
            ni = near_ir.reshape(256, 256, 1)
            lc = landcover.reshape(256, 256, 1)
            alt = altitude.reshape(256, 256, 1)

            patch1 = np.concatenate((ni, lc, alt), axis=2)

  
            #cs = MinMaxScaler()
            #print("PATCH GENERATOR")
            #print((df_env.loc[self.obs_ids[i]].values).shape)
            #print(cs.fit_transform(df_env.loc[self.obs_ids[i]].values).shape)
            #k = input()
            X_env_batch.append(df_env.loc[self.obs_ids[i]].values)
            #X_env_batch.append(df_env[self.obs_ids[i], :])
            #X_env_batch.append(cs.fit_transform(df_env.loc[self.obs_ids[i]].values.reshape(-1,1)))
            X_batch_rgb.append(rgb)
            X_batch_images.append(patch1)
            y_batch.append(self.labels[i])

            



        with self.lock:
        
            
            #print(len(X_batch_images))
            #print(len(X_batch_rgb))
            #print(np.asarray(X_batch_images).shape)
            #print("AH")
            #return {'input_1': np.asarray(X_batch), 'input_2': np.asarray(X_env_batch)}, np.asarray(np.array(y_batch))
            #return np.asarray(X_batch), np.array(y_batch)
            return (np.asarray(X_batch_rgb), np.asarray(X_batch_images), np.asarray(X_env_batch)), np.array(y_batch)

#### Pre-Trained ResNet-50

In [ ]:
#FROM https://chroniclesofai.com/transfer-learning-with-keras-resnet-50/
#for layer in res_img.layers:
from tensorflow.python.ops.image_ops_impl import rgb_to_grayscale
num_classes = len(set(df_obs['species_id']))
#pre_trained_res = Sequential()



#        layer.trainable=False

def transfer_learning(input_shape1, input_shape2, input_shape3, num_classes):
    #print("MADE IT")
    #image = tf.image.resize(image, (INP_SIZE[0], INP_SIZE[1]))

    res_img = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    #input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='none',
    classes=num_classes,
    )
    
    #print(input_shape1)
    #print(input_shape2)
    for layer in res_img.layers:
      layer.trainable = False
    rgb_input = tf.keras.layers.Input(input_shape1)
    #print("A")
    img_input = tf.keras.layers.Input(input_shape2)
    rgb = tf.keras.layers.Conv2D(3, kernel_size=(33, 33), activation='relu')(rgb_input)
    scale_layer = tf.keras.layers.Rescaling(scale=1 / 255, offset=-1)
    rgb = scale_layer(rgb)  
    #img = tf.keras.layers.Conv2D(8, kernel_size=(33, 33), activation='relu')(img_input)
    #print(rgb)
    tab_input = tf.keras.layers.Input(input_shape3)
    #print("B")
    rgb = res_img(rgb, training = False)
    #rgb = tf.keras.layers.GlobalAveragePooling2D()(rgb)
    #rgb = tf.keras.layers.Conv2D(3, (3,3), strides = 32)(rgb)
  #print("C")
    #rgb = tf.keras.layers.BatchNormalization()(rgb)
    #rgb = tf.keras.layers.Activation('relu')(rgb)
    #rgb = tf.keras.layers.Conv2D(3, (9,9), strides = 2, padding = "same")(rgb)
  #print("C")
    #rgb = tf.keras.layers.BatchNormalization()(rgb)
    #rgb = tf.keras.layers.Activation('relu')(rgb)
    #rgb = tf.keras.layers.Conv2D(3, (9,9), strides = 2, padding = "same")(rgb)
  #print("C")
    #rgb = tf.keras.layers.BatchNormalization()(rgb)
    #rgb = tf.keras.layers.Activation('relu')(rgb)
    #print("C")
    #rgb = tf.keras.layers.Flatten()(rgb)
    #print("D")
    img = img_input
    #img = res_img(img_input, training = False)
    rgb = tf.keras.layers.Dense(32, activation = 'relu')(rgb)
    rgb = tf.keras.layers.Flatten()(rgb)

    
    #rgb = tf.keras.layers.GlobalAveragePooling2D()(rgb)
    #rgb = tf.keras.layers.Dropout(0.2)(rgb)  # Regularize with dropout
    

    #img = tf.keras.layers.GlobalAveragePooling2D()(img)
    #img = tf.keras.layers.Dropout(0.2)(img)  # Regularize with dropout

    #z = tf.keras.layers.Concatenate(axis=1)([rgb, img])

    classifier = tf.keras.layers.Dense(num_classes, activation='softmax')(rgb)

  #classifier = tf.keras.layers.Dense(num_classes, name='outputs', activation='softmax')(x)
    model = tf.keras.models.Model(inputs = [rgb_input, img_input, tab_input], outputs = classifier, name = "pre-t-all-imag")
    #for layer in res_img.layers:
    #    layer.trainable=False
    model.summary()
    return model

    




CNN Multimodal

In [ ]:
def multi_modal_cnn():
    #with mirrored_strategy.scope():

        # Inputs
        patch_input = tf.keras.layers.Input(shape=(256, 256, 6), dtype='float32')
        tabular_input = tf.keras.layers.Input(shape=(29), dtype='float32')  

        # Augment data
    #     augmented = data_augmentation_for_visualization(patch_input)

        # From Scratch model
        x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(patch_input)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
        x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
        x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((1, 1), padding='same')(x)
        x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
        x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)      

        # Add Dense layers for images
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(256, activation='relu')(x)

        # Add Dense layers for Tabular data
        y = tf.keras.layers.Dense(512, activation='relu')(tabular_input)
        y = tf.keras.layers.Dense(256, activation='relu')(y)

        # Concatenate Image and tabular weights
        z = tf.keras.layers.Concatenate(axis=1)([x, y])

        # Add Classification Head
        z = tf.keras.layers.Dense(128, activation='relu')(z)
        classifier = tf.keras.layers.Dense(num_classes, name='outputs', activation='softmax')(z)

        # Define inputs and outputs
        model = tf.keras.Model(inputs=[patch_input, tabular_input], outputs=classifier)

        # Optimizer
        optimizer = tfa.optimizers.AdamW(learning_rate=0.001, 
                                         weight_decay=weight_decay_rate)
        # Compile model
        model.compile(optimizer=optimizer,
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[
                          tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                          tf.keras.metrics.SparseTopKCategoricalAccuracy(10, name="top-10-accuracy")
                      ]
                      )

#### For Images (Includes RGB, near IR, landcover, altitude) + Tabular (environmental vectors)

In [ ]:
BATCHSIZE = 128
train_generator = Environmental_Patches_Generator(obs_id_train, y_train, BATCHSIZE)
print("HERRE")
val_generator = Environmental_Patches_Generator(obs_id_val, y_val, BATCHSIZE)

# converting our train dataset to tf.data.Dataset
tf_train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator ,  # Our generator 
    output_types = ({'input_1': tf.float32 , 'input_2': tf.float32}, tf.float32) , # How we're expecting our output dtype
#    output_shapes = ({'input_1': [BATCH_SIZE, 256 , 256, 6], 'input_2': [BATCH_SIZE, 29]} , [BATCH_SIZE, ]) # How we're expecting our output shape
)

tf_val_dataset = tf.data.Dataset.from_generator(
    lambda: val_generator , 
    output_types = ({'input_1': tf.float32 , 'input_2': tf.float32}, tf.float32),
#    output_shapes = ({'input_1': [BATCH_SIZE, 256 , 256, 6], 'input_2': [BATCH_SIZE, 29]} , [BATCH_SIZE, ]) 
)



optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
num_classes = len(set(df_obs['species_id']))

input_shape1 = (256, 256, 6)
input_shape2 = (27)
model = my_res(input_shape1, input_shape2, num_classes)
# Compile model
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[
                  tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                  tf.keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy")
              ]
              )


HERRE


#### Pre-Trained All Images Code

In [ ]:
BATCHSIZE = 64
train_generator = Pretrained_Patches_Generator(obs_id_train, y_train, BATCHSIZE)
print("HERRE")
val_generator = Pretrained_Patches_Generator(obs_id_val, y_val, BATCHSIZE)

# converting our train dataset to tf.data.Dataset
#tf_train_dataset = tf.data.Dataset.from_generator(
#    lambda: train_generator ,  # Our generator 
#    output_types = ({'input_1': tf.float32 , 'input_2': tf.float32, 'input_3': tf.float32}, tf.float32) , # How we're expecting our output dtype
#    output_shapes = ({'input_1': [BATCH_SIZE, 256 , 256, 6], 'input_2': [BATCH_SIZE, 29]} , [BATCH_SIZE, ]) # How we're expecting our output shape
#)

#tf_val_dataset = tf.data.Dataset.from_generator(
#    lambda: val_generator , 
#    output_types = ({'input_1': tf.float32 , 'input_2': tf.float32, 'input_3': tf.float32}, tf.float32),
#    output_shapes = ({'input_1': [BATCH_SIZE, 256 , 256, 6], 'input_2': [BATCH_SIZE, 29]} , [BATCH_SIZE, ]) 
#)


#PROBLEM
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
num_classes = len(set(df_obs['species_id']))
#num_classes = 10
input_shape1 = (256, 256, 3)
input_shape2 = (256, 256, 3)
input_shape3 = (27)
model1 = transfer_learning(input_shape1, input_shape2, input_shape3, num_classes)
# Compile model
model1.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[
                  tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                  tf.keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy")
              ]
              )


HERRE
Model: "pre-t-all-imag"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_226 (InputLayer)         [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_91 (Conv2D)             (None, 224, 224, 3)  9804        ['input_226[0][0]']              
                                                                                                  
 rescaling_9 (Rescaling)        (None, 224, 224, 3)  0           ['conv2d_91[0][0]']              
                                                                                                  
 resnet50 (Functional)          (None, 7, 7, 2048)   23587712    ['rescaling_9[

#### For Environmental Vectors

In [ ]:
df_env = pd.read_csv("./geolifeclef-2022-lifeclef-2022-fgvc9/pre-extracted/environmental_vectors.csv", sep=";", index_col="observation_id")

X_train = df_env.loc[obs_id_train].values
X_val = df_env.loc[obs_id_val].values
X_test = df_env.loc[obs_id_test].values

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
extractor = PatchExtractor("./geolifeclef-2022-lifeclef-2022-fgvc9/pre-extracted/rasters", size=256)
extractor.add_all_bioclimatic_rasters()


#### Run Model

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.001, patience=10, 
                                            verbose=0, mode='auto', baseline=None, restore_best_weights=True)
train_results = model.fit(
    #train_ds,
    train_generator,
    validation_data = val_generator,
    #tf_train_dataset,
    #validation_data= val_ds,
    #validation_data = tf_val_dataset,
    epochs = 100,
    callbacks=[early_stop])
#)
# callbacks=[early_stop])

NameError: ignored

### Run Model Pretrained

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.001, patience=10, 
                                            verbose=0, mode='auto', baseline=None, restore_best_weights=True)
train_results = model1.fit(
    #train_ds,
    train_generator,
    validation_data = val_generator,
    #tf_train_dataset,
    #validation_data= val_ds,
    #validation_data = tf_val_dataset,
    epochs = 100,
    callbacks=[early_stop])
#)
# callbacks=[early_stop])

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


  266/24804 [..............................] - ETA: 3:38:42 - loss: 8.7786 - accuracy: 0.0031 - top-5-accuracy: 0.0150

KeyboardInterrupt: ignored

### Test Submission

In [ ]:
test_generator = Test_Patches_Generator(obs_id_test, BATCHSIZE)

tf_test_dataset = tf.data.Dataset.from_generator(
    lambda: test_generator,  # Our generator 
    output_types = (tf.float32 , tf.float32), # How we're expecting our output dtype
    #output_shapes = ([BATCHSIZE, 256 , 256, 3] , [BATCHSIZE, ]) # How we're expecting our output shape
    #output_shapes = (tf.TensorShape(features_shape), [BATCHSIZE, ]),
)

In [ ]:
import os
SUBMISSION_PATH = Path("submissions")
os.makedirs(SUBMISSION_PATH, exist_ok=True)


In [ ]:
from GLC.submission import generate_submission_file


n_test = len(df_obs_test)
s_pred = model.predict_generator(test_generator)

# Generate the submission file


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """


NameError: ignored

In [ ]:
print(s_pred[11])

NameError: ignored

In [ ]:
generate_submission_file("./submissions/new", df_obs_test.index, s_pred)